In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

In [8]:
df = pd.read_csv('standings2.csv')
df.head()

,Team,Balancing,Halfpipe,Funnel Endurance,Newton's Cradle,Long Jump,5 Meter Hurdles,Block Pushing,Relay Run,Event 9,Event 10,Event 11,Event 12,Event 13,Event 14,Event 15,Event 16
0,Minty Maniacs,25,15,25,10,8,8,,,,,,,,,,NaN
1,O'rangers,6,25,20,25,4,10,,,,,,,,,,NaN
2,Crazy Cat's Eyes,11,20,10,12,25,12,,,,,,,,,,NaN
3,Raspberry Racers,20,7,7,15,1,25,,,,,,,,,,NaN
4,Balls of Chaos,12,3,11,3,20,1,,,,,,,,,,NaN
